# Graphset Construction

### Useful references

* [Graph Automorphisms](https://en.wikipedia.org/wiki/Graph_automorphism)
* [Counting Graph Automorphisms](https://www.cs.umd.edu/~gasarch/papers/numauto.pdf)

In [5]:
from graph import Graph

import polars as pl
import matplotlib.pyplot as plt

import timeit
from queue import LifoQueue
from typing import Callable
from copy import deepcopy


In [6]:
max_n = 8
frame_schema = {
    "n": pl.UInt64,
    "full_graphset_size": pl.UInt64,
    "linear_reduced_graphset_size": pl.UInt64,
    "tree_reduced_graphset_size": pl.UInt64,
    "linear_full_graphset_time_s": pl.Float64,
    "tree_full_graphset_time_s": pl.Float64,
    "linear_reduced_graphset_time_s": pl.Float64,
    "tree_reduced_graphset_time_s": pl.Float64,
}

new_rows = []
for n in range(3, max_n):
    new_rows.append([
        n, 2**(n * (n - 1) / 2), 0, 0, float('inf'), float('inf'), float('inf'), float('inf')
    ])

enumeration_data = pl.DataFrame(
    data=new_rows,
    schema=frame_schema,
    orient="row"
)

### Graphsets

* $n$ - number of vertices
* $G(n)$ - The set of all graphs of order $n$
* $H(n)$ - The set of reduced graphs of order $n$, where removed graphs are automorphic to another graph in $H(n)$
* $LG(n)$ - The set of all graphs of order $n$ constructed by linear enumeration
* $TG(n)$ - The set of all graphs of order $n$ constructed by binary tree enumeration
* $LH(n)$ - The set of reduced graphs of order $n$ constructed by linear enumeration
* $TH(n)$ - The set of reduced graphs of order $n$ constructed by binary tree enumeration

### Linear Enumeration of G(n)

1. Iterate through all possible graphs via their ID: $0$ -> $2^{n \times (n - 1) \div 2}$ 
2. Convert an ID into a graph ([see Graphs for details](./Graphs.ipynb))
3. Perform some work on graph


In [7]:
def enumerate_linear_G(n: int, work: Callable[[Graph], None]):
    for i in range(0, int(2**(n*(n-1)/2))):
        g = Graph.from_id(i, n)
        work(g)

In [8]:
linear_G = []
def construction(n: int):
    linear_G.append([])
    enumerate_linear_G(n, lambda g: linear_G[-1].append(g))

loop = 1
for n in range(3, max_n):
    try:
        result = timeit.timeit('construction(n)', globals=globals(), number=loop)
    except ValueError as err:
        print(err.args[0])
        result = float('inf')

    enumeration_data[n - 3, "linear_full_graphset_time_s"] = result


### Linear Enumeration of H(n)

Reductions:

1. Manually insert only one graph with one edge
    * Skip any work on the graphs that have one edge in normal enumeration
    * This only really works if the work being performed on each graph is significantly more expensive than the if conditional check, because it will still enumerate the graph
2. Manually insert only one graph with only missing one edge
    * Same caveats and steps as for only one edge

In [9]:
def enumerate_linear_H(n: int, work: Callable[[Graph], None]):
    """
        Optimised Linear G(n) construction by manually removing some automorphic graphs from the list
            * All graphs with a single edge (there are $nC2$ of them) are removed, and only one is inserted
            * All graphs will only missing a single edge (there are $nC2$) are removed, and only one is inserted
    """
    l = int(n*(n-1)/2)

    # Doing this seperately to avoid running log2 on i = 0
    zero_graph = Graph.from_id(0, n)
    work(zero_graph)

    # Adding the single edge graph manually, then just avoiding adding any graph with a single edge into the list further down
    one_edge_graph = Graph.from_id(1, n)
    work(one_edge_graph)

    # Adding the missing one edge graph manually
    missing_one_edge_graph = Graph.from_id(2**l - 2, n)
    work(missing_one_edge_graph)

    for i in range(1, int(2**l)):
        # If the graph will have a single edge i.e. binary string has a single 1
        # Or if the graph will have a single missing edge i.e. binary string has a single 0
        if i.bit_count() != 1 and i.bit_count() != l - 1:
            g = Graph.from_id(i, n)
            work(g)

In [10]:
linear_H = []
def construction(n: int):
    linear_H.append([])
    enumerate_linear_H(n, lambda g: linear_H[-1].append(g))

loop = 1
for n in range(3, max_n):
    try:
        result = timeit.timeit('construction(n)', globals=globals(), number=loop)
    except ValueError as err:
        print(err.args[0])
        result = float('inf')

    enumeration_data[n - 3, "linear_reduced_graphset_time_s"] = result
    enumeration_data[n - 3, "linear_reduced_graphset_size"] = len(linear_H[n - 3])

### Binary Tree Enumeration of G(n)

In [11]:
def enumerate_tree_G(n: int, work: Callable[[Graph], None]):
    stack = LifoQueue()
    stack.put([])

    while not stack.empty():
        item = stack.get()

        if len(item) < int(n*(n-1)/2):
            item_left_child = deepcopy(item)
            item_left_child.append('0')

            item_right_child = deepcopy(item)
            item_right_child.append('1')

            stack.put(item_right_child)
            stack.put(item_left_child)

        else:
            graph_id = int("".join(item), base=2)
            g = Graph.from_id(graph_id, n)
            work(g)

In [12]:
tree_G = []
def construction(n: int):
    tree_G.append([])
    enumerate_tree_G(n, lambda g: tree_G[-1].append(g))

loop = 1
for n in range(3, max_n):
    try:
        result = timeit.timeit('construction(n)', globals=globals(), number=loop)
    except ValueError as err:
        print(err.args[0])
        result = float('inf')

    enumeration_data[n - 3, "tree_full_graphset_time_s"] = result

### Binary Tree Enumeration of H(n)

Reductions:

1. Insert some initial semi-constructed graph strings into the stack
    * This will reduce the number of strings needed to be iterated through then if the string was build from scratch
    * Provable that it still enumerates all the automorphic graphs (ok but now go do it lmao)
    * The chosen strings are the following:
        1. $i = 0 \text{ to } n - 1$
        2. String $i$ is the graph where vertex $1$ has $i$ edges

A Binary Tree of depth $d$ has $2^d - 1$ vertices. 
A string of length $n$ is at depth $n$ in the tree, so in the tree-enumeration of $G(n)$, the stack has $2^n - 1$ items.
The above reduction instead has $n$ strings in the stack.
Each stack item acts as a binary subtree with a depth of $n\times (n - 1) \div 2 - (n - 1)$ so, $|H(n)| = n2^{n\times (n - 1) \div 2 - n + 1}$

$$
    \text{Compression Ratio} = \frac{|G(n)|}{|H(n)|} 
    = \frac{2^{n\times (n - 1) \div 2}}{n2^{n\times (n - 1) \div 2 - n + 1}} 
    = \frac{2^{n\times (n - 1) \div 2}}{n2^{n\times (n - 1) \div 2}\cdot2^{1 - n}}
    = \frac{1}{n2^{1 - n}}
    = \frac{2^{n - 1}}{n}
$$

In [13]:
def enumerate_tree_H(n: int, work: Callable[[Graph], None]):
    stack = LifoQueue()

    for i in range(n):
        new_item = []
        for _ in range(i):
            new_item.append("1")
        for _ in range(n - 1 - i):
            new_item.append("0")

        stack.put(new_item)

    while not stack.empty():
        item = stack.get()

        if len(item) < int(n*(n-1)/2):
            item_left_child = deepcopy(item)
            item_left_child.append('0')

            item_right_child = deepcopy(item)
            item_right_child.append('1')

            stack.put(item_right_child)
            stack.put(item_left_child)

        else:
            graph_id = int("".join(item), base=2)
            g = Graph.from_id(graph_id, n)
            work(g)

In [14]:
tree_H = []
def construction(n: int):
    tree_H.append([])
    enumerate_tree_H(n, lambda g: tree_H[-1].append(g))

loop = 1
for n in range(3, max_n):
    try:
        result = timeit.timeit('construction(n)', globals=globals(), number=loop)
    except ValueError as err:
        print(err.args[0])
        result = float('inf')

    enumeration_data[n - 3, "tree_reduced_graphset_time_s"] = result
    enumeration_data[n - 3, "tree_reduced_graphset_size"] = len(tree_H[n - 3])

### Validation

Checking some properties that should have occurred for the above enumerations to be correct

1. $|LG(n)| = |TG(n)|$
2. $|LH(n)| < |G(n)|$
2. $|TH(n)| < |G(n)|$

In [15]:
for n in range(3, max_n):
    i = n - 3

    # TG(n) == LG(n)
    try:
        assert(len(tree_G[i]) == len(linear_G[i]))
    except AssertionError:
        print(f"|LG({n})| != |TG({n})| -> {len(linear_G[i])} != {len(tree_G[i])}")

    # LH(n) < G(n)
    try:
        assert(len(linear_H[i]) < len(linear_G[i]))
    except AssertionError:
        print(f"|LH({n})| >= |LG({n})| -> {len(linear_H[i])} >= {len(linear_G[i])}")

    # TH(n) < G(n)
    try:
        assert(len(tree_H[i]) < len(tree_G[i]))
    except AssertionError:
        print(f"|TH({n})| >= |G({n})| -> {len(tree_H[i])} >= {len(tree_G[i])}")

In [16]:
enumeration_data.write_csv("./data/graphset_enumeration.csv")
enumeration_data

n,full_graphset_size,linear_reduced_graphset_size,tree_reduced_graphset_size,linear_full_graphset_time_s,tree_full_graphset_time_s,linear_reduced_graphset_time_s,tree_reduced_graphset_time_s
u64,u64,u64,u64,f64,f64,f64,f64
3,8,4,6,0.001319,0.00115,0.000885,0.000932
4,64,54,32,0.00797,0.008272,0.003668,0.00352
5,1024,1006,320,0.108147,0.19817,0.180353,0.046388
6,32768,32740,6144,3.17081,4.30892,3.149772,0.809325
7,2097152,2097112,229376,231.376663,316.855455,232.943607,34.593764
